# ibm_sql_query

Execute arbitrary SQL queries against CSV and PARQUET files using IBM Cloud SQL Query and Cloud Object Store

In [ ]:
!pip3 install ibmcloudsql

In [ ]:
import glob
import logging
import ibmcloudsql
from ibmcloudsql import SQLQuery
import os
import shutil
import sys
import re

In [ ]:
# IBM Cloud API key
api_key = os.environ.get('api_key')

# URI of resulting file (example: cos://s3.eu-de.cloud-object-storage.appdomain.cloud/cos-rkie-sqlquery-test/result)
result_file_uri = os.environ.get('result_file_uri')

# (unique) Custom Resource Name (CRN) of IBM SQL Query Service
sql_query_crn = os.environ.get('sql_query_crn')

# sql statement to execute ()
sql = os.environ.get('sql')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
sqlClient = SQLQuery(api_key, sql_query_crn)
sqlClient.run_sql(sql + ' INTO {} STORED AS CSV'.format(result_file_uri))